# scikit-learn (sklearn) で線形回帰（重回帰）を動かす
**scikit-learn (sklearn)はPythonの機械学習ライブラリ**です．公式：https://scikit-learn.org/stable/

sklearnを用いると，驚くべきことに，**ライブラリのインポート・データの読み込みを除けば10行未満で学習・予測**まで行うことができます．
Pythonなので遅いのではないか（自分でCで書いた方が速いのではないか）と思うかもしれませんが，sklearnではNumPyやSciPyを上手く用いて書かれたコードや，Cで書かれた大変質の良いコードが動いているため，sklearnはかなり高速に動作します．
ほとんどの場合，雑に書いたCのプログラムよりは高速に動作します．
またPythonに慣れていなくとも，「慣れていないPythonでscikit-learnを使えるようになる時間」と「慣れているCで実装する時間」だと，おそらく圧倒的に前者の方が短いでしょう．

勿論，勉強のためにsklearnを使わずに実装しても構いませんが，本演習ではデータ分析全体の流れを体験してもらいたいため，サンプルコードではsklearnを用います．

## 今回学ぶこと
- pandasによるcsvの読み込みと簡単な操作（列・行へのアクセス，`numpy.ndarray`への変換）
- sklearnの基本的な使い方
- 提出までの流れ：
  1. 特徴ベクトルの作成
  2. モデル・アルゴリズムの選定，学習
  4. 予測
  5. 提出

## 予測モデル構築の流れ

本演習のように，機械学習を用いて予測モデルを構築し，未知の（テスト）データに対して予測を行う手順は，主に以下のようになります．

1. データを用意し，特徴ベクトルを作る
2. どのような手法（モデル）を使うかを決める
3. モデルを学習する方法を決め，学習する
4. 未知のデータに予測を行う（そして本演習では提出する）


## データ読み込み
演習で用いるデータはcsvファイル（comma-separated valuesファイル）で表現されています．
Pythonの標準の機能を使って読み込んでも良いのですが，Pythonには**pandasというデータ解析のための便利なライブラリ**があります．
pandasによって，CSVを含む様々なファイルを簡単に読み込み，さらに読み込んだデータに対して様々な処理を簡単に行うことができます．

なお，今回は文字列の情報を含んでいるためpandasを用いますが，数値情報だけであれば，Pythonにおける**数値・行列・線形代数計算ライブラリのNumPy**で読み込んで処理するほうが楽であることも多いです（NumPyについては，day1_2_how_to_numpy.ipynbを参照）．
また，scikit-learnの関数には，NumPyの配列を前提とするものやNumPyの配列を返すものも数多くあります．そのため，今回の演習ではpandasで最低限の処理をした後，NumPyの配列に変換し（=いろいろな演算がサポートされている便利な配列），変換したものをscikit-learnに渡すことにします．
つまり，以下のような手順を行います：
1. `pandas`でデータを読み込む
2. `pandas`の便利な関数でごにょごにょして，`numpy`の`ndarray`（`array`）に変換
3. （場合によっては）`numpy`の便利な関数でごにょごにょして，特徴ベクトルを作成
4. scikit-learnに渡して学習

まず，今回使うものを`import`（=C言語で言うところのincludeだと思ってください）をしましょう．

In [33]:
import numpy as np
import pandas as pd
# 機械学習のライブラリ．今回は線形回帰LinearRegressionを動かす．
# LinearRegressionはsklearnのlinear_modelモジュールの中にあるので，次のようにインポートする
from sklearn.linear_model import LinearRegression

上のセルを動かしましたか？その際に特にエラーが出なければ，インポートに成功しています．
これ以降皆さんはnumpy，pandas，そしてsklearn.linear_modelの`LinearRegression`が使えるようになりました！


では，次は訓練データとテストデータを読み込みます．
csvファイルは，pandasの`read_csv`という関数を使って読み込むことができます．
ファイルのパスを引数として渡すと，`pandas.DataFrame`という型で（オブジェクト指向言語なので，「`pandas.DataFrame`クラスのインスタンスを作成して」とも言える）読み込んだものを返してくれます．
詳細は[公式のドキュメント](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)を参照してください．
では実際に動かしてみます．

このノートブックを動かしているディレクトリにdataディレクトリがあり，その中にtrain.csvとtest.csvが存在していると仮定しています．
そうでない場合は適宜書き換えてください．

Goole Colabを用いている場合，Google Driveにdataフォルダを置いて，次のセルのコメントアウトを外して動かしてください（ローカル環境で動かしている場合は，次のセルは飛ばして良いです）．次のセルを動かした後，以下の手順を踏むことでGoogle Driveのデータを読み込むことができるようになります．
- 以下のセルを動かすと，URLが出てくるのでそこにアクセスしてください．
- Googleアカウントのログインが求められるので，ログインしてください．
- authorization codeが表示されるので，それを貼り付けてください．
- 更にその下のセルの`read_csv`で指定されているパスを適宜書き換えてください．画面左側のバーで「フォルダ」のタブを選ぶと"drive"というフォルダが出てくると思いますが，そこがGoogle Driveを表しています．そこを見て適宜パスを書き換えてください．

In [34]:
# Google Colabを用いる場合、以下のコメントアウトを外して実行
# from google.colab import drive
# drive.mount('/content/drive') # google driveをマウント（＝Colabから使えるようにする）

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
# Google Colabを用いる場合、以下のコメントアウトを外して実行
# d_train = pd.read_csv("drive/My Drive/data/train.csv") # 訓練データを読み込む．パスは適宜変更すること
# d_test = pd.read_csv("drive/My Drive/data/test.csv") # テストデータを読み込む．パスは適宜変更すること

In [ ]:
# ローカル環境を用いる場合、以下のコメントアウトを外して実行
# d_train = pd.read_csv("/data/train.csv") # 訓練データを読み込む．パスは適宜変更すること
# d_test = pd.read_csv("/data/test.csv") # テストデータを読み込む．パスは適宜変更すること

エラーが何も出ていなければ読み込めています．しかし言われるがまま読み込んだだけでどうなっているのかよくわかりませんね．`print`してみましょう．

In [47]:
print("訓練データ")
print(d_train)
print("\nテストデータ")
print(d_test)

訓練データ
                     track_id  \
0      2Fc4CqCia3MvJeWULkzW9o   
1      1Mf4ZMtbgDrFYvv1QZ69gm   
2      0nejfXN5tjdICWIdfakXWx   
3      4qcyHRRheEFLxnkLh29hrY   
4      3lWuagNhAzcqOikD5KnBku   
...                       ...   
79795  0ojGXbSAH6INkZX24Kfs2u   
79796  3HmUz6Dnk7PTfEs6OhJvlN   
79797  2vX97IafvBPXQ6xUTAWErl   
79798  5dqWZZQmtkuQtwbumKVdA7   
79799  3i39d7KtpiAHW5elg1WupN   

                                                 artists  \
0                                        Olivier Abeille   
1                                               Nonpoint   
2                             Porter Robinson;Urban Cone   
3                                                 Rosana   
4                                    The Everly Brothers   
...                                                  ...   
79795                                             Accept   
79796           Andrew Lloyd Webber;Carrie Hope Fletcher   
79797                                               Kato 

train.csvの中身が出てきました！すばらしいですね．
詳細はスライドを確認してほしいですが，この行列は，各行が一つの楽曲のデータに対応しています．
各列は楽曲の何らかの情報を表していて，各列がどのような情報なのかはヘッダーに書かれています．
たとえば，1列目は"track_id"と書かれているので，これは楽曲のトラックのID情報ですね．
訓練データの最後の列は"popularity"と書かれており，これはまさしく目標値（出力・教師情報）です．
テストデータに"popularity"の列はなく，この列の値を予測して，その予測の正確さで皆さんに競っていただくのがこのコンペティションです．


さて，予測モデルを動かす前に，ちょっと`d_train`で遊んでみます．各列や行を取り出してみましょう．

列を取り出すのは非常に簡単で，他のプログラミング言語で配列の要素にアクセスするのと同じようにブラケット（角括弧）を用います．具体的には，

d_train[列の名前]

とすることで取り出せます．また，複数の列を取り出すことは，

d_train[列の名前のリスト]

でできます（"取り出す"と言っていますが，元の`DataFrame`から消えるわけではありません）．以下のセルで実際にやってみましょう．

In [48]:
print("人気度の列を取り出す")
print(d_train['popularity'])

print("\n楽曲のジャンルとテンポを取り出す")
print(d_train[["track_genre", "tempo"]])

print("\n取り出した結果を別の変数に格納することもできる")
d_train_key = d_train["key"]
print(d_train_key)


人気度の列を取り出す
0        43
1         0
2        50
3        41
4        58
         ..
79795    55
79796     0
79797    60
79798    44
79799    24
Name: popularity, Length: 79800, dtype: int64

楽曲のジャンルとテンポを取り出す
       track_genre    tempo
0            study   76.972
1       industrial   99.956
2          dubstep  119.993
3            r-n-b  130.802
4      rock-n-roll   97.166
...            ...      ...
79795  heavy-metal  167.973
79796   show-tunes   71.612
79797        anime   56.997
79798       pagode   99.964
79799   show-tunes   87.218

[79800 rows x 2 columns]

取り出した結果を別の変数に格納することもできる
0         8
1         0
2         8
3        10
4         2
         ..
79795    11
79796     9
79797     4
79798     2
79799     1
Name: key, Length: 79800, dtype: int64


とりだせました！すばらしいですね．

次に行を取り出してみましょう．行も列と同様にブラケット（角括弧）で取り出せます．d_train\[i:j\]とすることで，i行目からj-1行目の値を取り出すことができます．

In [49]:
print("0から4-1行目まで取り出す")
print(d_train[0:4])
print("\n10行目だけ取り出す")
print(d_train[10:11])

0から4-1行目まで取り出す
                 track_id                     artists     album_name  \
0  2Fc4CqCia3MvJeWULkzW9o             Olivier Abeille  Notes & Notes   
1  1Mf4ZMtbgDrFYvv1QZ69gm                    Nonpoint          Metal   
2  0nejfXN5tjdICWIdfakXWx  Porter Robinson;Urban Cone         Worlds   
3  4qcyHRRheEFLxnkLh29hrY                      Rosana       Momentos   

           track_name  duration_ms  explicit  danceability  energy  key  \
0             Rooftop       126727     False         0.630   0.225    8   
1  In the Air Tonight       271240     False         0.562   0.808    0   
2         Lionhearted       266106     False         0.621   0.740    8   
3        Nem um Toque       307800     False         0.563   0.415   10   

   loudness  mode  speechiness  acousticness  instrumentalness  liveness  \
0   -17.199     0       0.0897        0.9020          0.900000    0.1170   
1    -4.836     1       0.0462        0.0118          0.158000    0.1370   
2    -6.925     1   

できました．素晴らしいですね！他にも色々できますし，同じことを行う別の方法があったりもしますが，今回はとりあえずこの程度にしておきましょう．
詳細は例えばhttps://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html 等を見てみてください．

さて，次に特徴ベクトルを作っていきたいのですが，その前にいくつかの変数を準備します．

まず，訓練データとテストデータの数を表す変数`n_train`と`n_test`を準備します．今日は使いませんが，今後よく使うためここで取得の仕方を確認しておきます．
各行が一つのデータに対応しているので，**`d_train`や`d_test`の行の数**が取得できればよいのですが，これは`len`関数によって取得できます．

また，sklearnでモデルを学習させる際に必要となるので，訓練データの目標値"popularity"を`d_train`から取り出して分離させておきます．
`pop`メソッドを使うことで，`DataFrame`（`d_train`や`d_test`）から特定の列を分離させることができます．
「分離」と言っている通り，ブラケットによるアクセスと違い，`pop`で取り出すと**元の`DataFrame`からその列は削除されます**．

In [50]:
print("訓練データとテストデータの数を取得")
n_train = len(d_train)
n_test = len(d_test)
print(f"訓練データ数：{n_train}，テストデータ数：{n_test}")
print("\n 出力情報を取り出す．popularityの列が消えている")
# targetの値
y_train = d_train.pop('popularity')
print(d_train)
print(y_train)
print(f"y_trainのクラスは{type(y_train)}")

訓練データとテストデータの数を取得
訓練データ数：79800，テストデータ数：34200

 出力情報を取り出す．popularityの列が消えている
                     track_id  \
0      2Fc4CqCia3MvJeWULkzW9o   
1      1Mf4ZMtbgDrFYvv1QZ69gm   
2      0nejfXN5tjdICWIdfakXWx   
3      4qcyHRRheEFLxnkLh29hrY   
4      3lWuagNhAzcqOikD5KnBku   
...                       ...   
79795  0ojGXbSAH6INkZX24Kfs2u   
79796  3HmUz6Dnk7PTfEs6OhJvlN   
79797  2vX97IafvBPXQ6xUTAWErl   
79798  5dqWZZQmtkuQtwbumKVdA7   
79799  3i39d7KtpiAHW5elg1WupN   

                                                 artists  \
0                                        Olivier Abeille   
1                                               Nonpoint   
2                             Porter Robinson;Urban Cone   
3                                                 Rosana   
4                                    The Everly Brothers   
...                                                  ...   
79795                                             Accept   
79796           Andrew Lloyd Webber;Carrie Hope

訓練データ数は`79800`，テストデータ数は`34200`で，これはスライドやコンペティションのページに書いてあるとおりです．
また，`d_train`から"popurality"の列が消えていて，`y_train`が"popurality"の情報を持っていますね．

今，`y_train`は`Series`クラスのインスタンスになっていますが，後々の都合のためNumPyの配列への変換を考えます．
便利なことに，`y_train.to_numpy()`とすることで列の名前や行の番号が消え，実際に持っている値だけをNumPyの配列として取り出すことができます．

ちなみに，`DataFrame`クラスのインスタンス`d_train`に対して`d_train.to_numpy()`とすることでも，同じく行番号や列の名前を消して中の値だけをNumPyの配列として取り出すことができます．

In [51]:
y_train = y_train.to_numpy()
print(y_train)
print(f"y_trainのクラスは{type(y_train)}")

[43  0 50 ... 60 44 24]
y_trainのクラスは<class 'numpy.ndarray'>


### ここまでのまとめ
- `pandas`の`read_csv`でcsvファイルを読み込むことができる．読み込んだファイルは`DataFrame`クラスのインスタンスとして返される．
- `DataFrame`クラスのインスタンスは`print`で中身を簡単にわかりやすく見ることができる．
- `DataFrame`では，[列の名前]や[列のリスト]とすることで，特定の列を取り出すことができる．
- `DataFrame`では，[i:j]とすることで，特定の範囲の行を取り出すことができる．
- `.to_numpy()`で`DataFrame`や`DataFrame`から取り出した列の持つ値を，NumPyの配列`np.array`として取り出すことができる．

## 特徴ベクトルを作る

さて，いよいよ特徴ベクトルを作っていきましょう．訓練データにおける入力の情報として，以下が与えられています：

**文字列情報**
- track_id: 楽曲トラックに付けられるID
- artists: 演奏したアーティストの名前(複数人の場合は;で区切られる)
- album_name: 収録されているアルバム名
- track_name: 楽曲トラックの名前
- track_genre: 楽曲の属するジャンル

**数値(連続値)情報**
- duration_ms: 楽曲の長さ(ミリ秒)
- danceability: ダンスに適しているかの指標．テンポ、リズムの安定性、ビートの強さ、全体的な規則性などの音楽的要素の組み合わせに基づいて計算される
- energy: エネルギッシュさの感覚的な指標
- loudness: 楽曲全体の音量(dB)
- speechiness: 楽曲の中でどれだけ発話されているかの指標．値が高いほど発話が多い
- acousticness: 楽曲がアコースティック(電気信号を用いない)であるかの指標．値が高いほどアコースティックである可能性が高い
- instrumentalness: 楽曲にボーカルが含まれていないかの指標．値が高いほどボーカルが含まれていない可能性が高い
- liveness: 楽曲がライブ演奏であるかどうかの指標．値が高いほどそのトラックがライブ演奏である可能性が高い
- valence: 楽曲が伝える感情のポジティブさの指標．値が高いほどポジティブな，低いほどネガティブな感情のイメ―ジを伝える
- tempo: 楽曲の推定BPM(1分当たりの拍数)

**数値(離散値)情報**
- key: キー(音の高さ)
- mode: 楽曲の長調/短調の区別(長調は1, 短調は0)
- time_signature: 推定される拍子記号を表す整数値．例えば5のとき楽曲が4分の5拍子であることを表す

**真偽値情報**
- explicit: 楽曲に歌詞があるか(True or False)

**ターゲット(予測対象)情報**
- popularity: 楽曲の人気度を表す0から100の整数値．100は最も人気があることを表す

上記に示したように，入力データには文字列の情報が含まれています．
しかし，機械学習のアルゴリズムの多くは各データが「数値のベクトル」として表されていると仮定しており，このような**文字列の情報はそのまま扱えません**．
今回動かしてみる線形回帰も，数値の情報しか扱えません．
機械学習や統計では，基本的には**一つ一つのデータは数値のベクトル**として表現され，**データの集まりは数の行列**として表現されている，と考えます．つまり，一つ一つの行が一つのデータに対応し，一つ一つの列が何らかの情報（特徴）に対応すると考えます．同様に，**訓練データの目標値（ここではpopurarity）の集まりは数のベクトル**として表現されていると考えます．
sklearnを使って学習する際も，訓練データの入力を表す行列と，訓練データの目標値を表すベクトルを渡す必要があります．

今回のノートブックでは最短で提出まで行うことを目標として，上に示した特徴量のうち**元から数値あるいは真偽値で表される情報だけ**を用いることにします．
文字列で表される情報は次回用いることにします（もちろん，どのように用いるかのアイデアがすでにあり，そのためのプログラムが書けそうであるならば，ぜひ行ってみてください）．

以下のコードでは，特徴量として入力する列を変数`columns_num`によって定義しています．

In [60]:
columns_num = ["duration_ms", "danceability", "energy", "key", "loudness", "mode","speechiness","acousticness","instrumentalness","liveness","valence","tempo","time_signature","explicit"]

X_train_num = d_train[columns_num].to_numpy()
X_test_num = d_test[columns_num].to_numpy()

これで変数`X_train_num`と`X_test_num`にはそれぞれ訓練データとテストデータの特徴ベクトル(の集まりからなる行列)が格納されたはずです．

ここで，変数`X_train_num`と`X_test_num`の型と配列の形状を確認してみましょう．
`X_train_num`の場合，型は`type(X_train_num)`，配列の形状は`X_train_num.shape`で取得できます．



In [61]:
print("X_train_numを表示")
print(type(X_train_num), X_train_num.shape)
print(X_train_num)
print("\nX_test_numを表示")
print(type(X_test_num), X_test_num.shape)
print(X_test_num)

X_train_numを表示
<class 'numpy.ndarray'> (79800, 14)
[[126727 0.63 0.225 ... 76.972 4 False]
 [271240 0.562 0.808 ... 99.956 4 False]
 [266106 0.621 0.74 ... 119.993 4 False]
 ...
 [77754 0.476 0.0792 ... 56.997 4 False]
 [197146 0.646 0.834 ... 99.964 4 False]
 [142480 0.185 0.0654 ... 87.218 3 False]]

X_test_numを表示
<class 'numpy.ndarray'> (34200, 14)
[[239893 0.85 0.66 ... 101.947 4 True]
 [209093 0.64 0.758 ... 106.978 4 False]
 [163235 0.712 0.347 ... 135.975 4 False]
 ...
 [647866 0.516 0.772 ... 160.048 4 False]
 [301346 0.649 0.76 ... 106.002 4 False]
 [240133 0.436 0.934 ... 91.481 4 False]]


`X_train_num`と`X_test_num`の中身を見てみると，数値に混じって`True`と`False`の値が入っています．これは真偽値情報である`explicit`をそのまま`numpy.ndarray`に変換したためです．今回の場合は予測を実行する際に`True`や`False`の値が入っていても大きな問題はありませんが，一応値を変換しておきましょう．`pandas.DataFrame`内で列ごとの型を変更するには`astype`を用います．

In [68]:
d_train = d_train.astype({'explicit': int}) # `explicit`の列の型をintにキャスト
d_test = d_test.astype({'explicit': int}) # `explicit`の列の型をintにキャスト

X_train_num = d_train[columns_num].to_numpy()
X_test_num = d_test[columns_num].to_numpy()

In [69]:
print("X_train_numを表示")
print(type(X_train_num), X_train_num.shape)
print(X_train_num)
print("\nX_test_numを表示")
print(type(X_test_num), X_test_num.shape)
print(X_test_num)

X_train_numを表示
<class 'numpy.ndarray'> (79800, 14)
[[1.26727e+05 6.30000e-01 2.25000e-01 ... 7.69720e+01 4.00000e+00
  0.00000e+00]
 [2.71240e+05 5.62000e-01 8.08000e-01 ... 9.99560e+01 4.00000e+00
  0.00000e+00]
 [2.66106e+05 6.21000e-01 7.40000e-01 ... 1.19993e+02 4.00000e+00
  0.00000e+00]
 ...
 [7.77540e+04 4.76000e-01 7.92000e-02 ... 5.69970e+01 4.00000e+00
  0.00000e+00]
 [1.97146e+05 6.46000e-01 8.34000e-01 ... 9.99640e+01 4.00000e+00
  0.00000e+00]
 [1.42480e+05 1.85000e-01 6.54000e-02 ... 8.72180e+01 3.00000e+00
  0.00000e+00]]

X_test_numを表示
<class 'numpy.ndarray'> (34200, 14)
[[2.39893e+05 8.50000e-01 6.60000e-01 ... 1.01947e+02 4.00000e+00
  1.00000e+00]
 [2.09093e+05 6.40000e-01 7.58000e-01 ... 1.06978e+02 4.00000e+00
  0.00000e+00]
 [1.63235e+05 7.12000e-01 3.47000e-01 ... 1.35975e+02 4.00000e+00
  0.00000e+00]
 ...
 [6.47866e+05 5.16000e-01 7.72000e-01 ... 1.60048e+02 4.00000e+00
  0.00000e+00]
 [3.01346e+05 6.49000e-01 7.60000e-01 ... 1.06002e+02 4.00000e+00
  0.00000e+

キャストによって`X_train_num`と`X_test_num`の中身が数値のみになりました！

ここまでのコードが正しく実行できていれば，上のセルの出力の2行目には

<class 'numpy.ndarray'> (79800, 14)

のように表示されていると思います．これは`X_train_num`が`numpy.ndarray`クラスのインスタンスであり，その形状が2次元配列で1次元目のサイズが79800，2次元目のサイズが14(端的に言えば79800行$\times$14列の行列)であることを表しています．

列の数が14ということは，各データが14個の数値的な情報によって表現されている，ということになります．
このようなデータを表す情報のことを**特徴(feature)**といい，特徴のベクトルのことをそのまま**特徴ベクトル**，特徴の数を**（データの，あるいは特徴の）次元**と言います．つまり，今は14次元の特徴ベクトルとしてデータを表現しているといえます．

## 線形回帰を動かす
ここまでおつかれさまでした．**ついに機械学習アルゴリズムを動かす**ときが来ました．
sklearnの教師あり学習のモデル・アルゴリズムの実装について，以下に列挙されています：

https://scikit-learn.org/stable/supervised_learning.html#supervised-learning

凄まじい量ですね．色々ありますが，最も単純な手法の一つである**線形回帰（重回帰）=LinearRegression**を動かしてみます．

一応，線形回帰について簡単に説明しておきます．まず，**線形モデル（Linear Model）**は以下のようなモデルです：

$$
 y(\mathbf{x}; \mathbf{w}) = \mathbf{w}^{\top} \mathbf{x} = \sum_{j=1}^{D}w_jx_j,
$$


ここで， $\mathbf{x} \in \mathbb{R}^{D}$ は特徴ベクトル， $D$ はその次元（今回の例だと$D=14$）， $\mathbf{w} \in \mathbb{R}^{D}$ は線形モデルの学習されるパラメータです（ $x_1$ や $x_2$ というのが一つの特徴を表しています）．
「**線形モデルの学習**」は「**訓練データから $\mathbf{w}$ をイイカンジに定める**」ことを意味します．
線形モデルを回帰に用いるとき，あるいは**平均二乗誤差**

$$
\frac{1}{N} \sum_{n=1}^N (y(\mathbf{x}_n; \mathbf{w})-t_n)^2
$$

を最小化することで学習を行うとき，線形モデルは特に**線形回帰**と呼ばれます( $t_n$ は訓練データにおける特徴ベクトル $\mathbf{x}_n$ に対応する出力を表します)．
データサイエンスの初回で導入された**多項式回帰**も結局のところ線形回帰とみなすことができます（多項式特徴ベクトルに対する線形回帰）．

**線形モデル**の学習アルゴリズムは一つではなく様々あります．
線形回帰では二乗誤差を最小化しましたが，別の関数を最小化しても良いです．
また，同じ関数を最小化する場合でも，最小化の仕方（アルゴリズム）も一つとは限りません．
また，「何らかの制約を課せられた線形モデル」というのもあったりします．
「二乗誤差を用いた線形モデル」が「線形回帰」と呼ばれるように，学習アルゴリズムが変わるとまた別の名前で呼ばれるようになったりします．
[先程のsklearnの教師あり学習のモデル・アルゴリズムの実装一覧](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning)を見ると，**Linear Models**に属している手法が**合計で17個**もありますが，これらは基本的には全てモデル（関数の形）としては同じだけれども，学習アルゴリズム等が違うため分けられています．


線形回帰の学習方法についてはデータサイエンスの授業で習い，実験で実際に定める（学習する）プログラムを書くことになりますが，今回の演習ではあくまでデータ分析全体の流れを理解することなので，学習するプログラムを皆さんに書いてもらうことはありません（もちろん，勉強のために書くことは素晴らしいことだと思いますが）．偉大なるsklearnが勝手にやってくれるのです．

ちょっと話が長くなってしまいました．本題に入ります．`LinearRegression`を使えば線形回帰を動かすことができます．
sklearnを用いる基本的な手順は，
1. モデルのインスタンスを作成
2. 作成したモデルオブジェクトを**`fit`メソッド**を用いて学習．`fit`メソッドには訓練データの入力と目標値（つまり，行列とベクトル）を渡す．
3. 学習したモデルを用いて**`predict`メソッド**で予測．`predict`メソッドにはデータの入力（つまり行列）を渡す．

となっています．スライドでは，プロセスの中に「学習アルゴリズムを選ぶ」というような手順が入っていたと思います．
sklearnでは学習アルゴリズムが違う場合は別のクラスとして提供されていることが多いです（先程の線形モデルの例のように）．
また，一つのクラスの中で学習アルゴリズムを選べる場合も，インスタンスを作成する時に学習アルゴリズムを指定します．
したがって，sklearnでは「モデルのインスタンスを作成」した時点で学習アルゴリズムを選んだことになります．


では実際に上の手順通りに学習してみます．
Pythonにおいて，`ClassA`という名前のクラスのインスタンスは`ClassA()`で作ることができるのでした（day1_1_how_to_python.ipynb参照）．
したがって，今回のケースでは以下のようになります．

In [70]:
# 手順1：LinearRegressionのインスタンスの作成
lr = LinearRegression()
# 手順2：上で作ったオブジェクトの学習
lr.fit(X_train_num, y_train)
# 手順3：テストデータに対する予測
y_pred_test = lr.predict(X_test_num)

**これで学習とテストデータに対する予測まで終わりました!**たった3行です．すごいですね．予測結果をプリントしてみましょう．

In [71]:
print(y_pred_test)

[37.76216776 31.28461239 37.10203066 ... 34.4882262  30.34860045
 32.95970719]


## 予測結果の提出

予測ができたので，この結果を提出することを考えます．予測結果をファイルに出力しなければなりません．ファイル出力ときくと面倒そうだと思うかもしれませんが，NumPyを用いれば以下のように簡単にできます．
`np.savetxt`の`X`という引数で保存したい配列を指定し，`fname`という引数でファイル名を指定します．
今回は，`y_pred_test`を`y_pred_lr.csv`という名前で保存することにします．

In [72]:
np.savetxt(X=y_pred_test, fname='y_pred_lr.csv')

y_pred_lr.csvというファイルが生成されました（Google Colabの場合は，画面左側のタブが「ファイル」のであるとき，「ファイル」という文字の下に3つアイコンがありますが，その真ん中の「更新」アイコンを押すと出てくるはずです．提出する際には生成されたy_pred_lr.csvをダウンロードしてください）．これを提出してみると，出力が全て0のy_pred_example.csvより良い結果になると考えられます（ちゃんと学習ができていれば）．

## 線形回帰の修正

線形モデルは以下の式

$$
 y(\mathbf{x}; \mathbf{w}) = \mathbf{w}^{\top} \mathbf{x} = \sum_{j=1}^{D}w_jx_j
$$

で表される，と述べました．あきらかに，どのような$\mathbf{w}$を推定（学習）しても， $\mathbf{x}=\mathbf{0}$ のとき予測結果は $0$ となってしまいます．
一次元の場合で考えるとわかりやすいです：このモデルは**原点を通る直線**しか表現することができません．
そこで，以下のように新たに**バイアス項（bias term）**あるいは**切片（intercept）**と呼ばれるパラメータ $b \in \mathbb{R}$ を導入することがあります：

$$
 y(\mathbf{x}; \mathbf{w}, b) = \mathbf{w}^{\top} \mathbf{x} + b= \sum_{j=1}^{D}w_jx_j + b.
$$

 $b$ も $\mathbf{w}$ と同様に学習するパラメータです．

 $b$ を導入したほうが良いのか否かはタスクやデータに依存しており，それ自体は学習するものではなく，**機械学習のユーザが決める**ものです．
機械学習のモデルや学習アルゴリズムの中には，このように**学習するのではなくユーザが定める要素**を持つものがあります（今回は「用いるか否か」ですが，連続値や離散値を定めないといけないこともある）．
このような要素（パラメータ）を**ハイパーパラメータ**と言います（もっとも，今回の例では「用いるか否か」であるためあまり**パラメータ感がない**上に，さらに $b=0$ と推定されれば「用いない」に対応するため，あまりハイパーパラメータとは感じない（もしかすると言わない）かもしれませんが）．
わかりやすいハイパーパラメータの例としては，データサイエンスの初回の授業で出てきた，**正則化項の強さ $\lambda$** があげられます．

さて，この話を聴くと **$b$ を導入するぞ！**と思うのではないでしょうか．
しかし，**実はsklearnの`LinearRegression`ではデフォルトで導入**されています．
そこで，（嬉しいかは置いておいて）**$b$ を導入しない**ことを考えてみます．
sklearnにおいて，モデルのハイパーパラメータはインスタンスを作る際に指定することができます．

[sklearnのLinearRegressionのドキュメント](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)を読むと，`fit_intercept`というパラメータがあり，"Whether to calculate the intercept for this model."と説明されています(みなさんも探してみてください)．`fit_intercept`はbool型の変数とあるので，`fit_intercept=False`を，インスタンスを作る時に指定してあげればよいです．

せっかくなので，この「バイアス項（切片）を使わない線形回帰」を用いた予測結果を`y_pred_lr_without_bias.csv`として保存してみましょう．（提出はしてもしなくてもよい）．

In [73]:
lr = LinearRegression(fit_intercept=False)
lr.fit(X_train_num, y_train)
y_pred_test_without_bias = lr.predict(X_test_num)
print(y_pred_test_without_bias)
np.savetxt(X=y_pred_test_without_bias, fname='y_pred_lr_without_bias.csv')

[38.07432601 32.28749487 35.69171582 ... 37.45991111 32.67283298
 31.15342993]


In [74]:
print("バイアス項ありでの予測結果：")
print(y_pred_test[:5])
print("バイアス項なしでの予測結果：")
print(y_pred_test_without_bias[:5])

バイアス項ありでの予測結果：
[37.76216776 31.28461239 37.10203066 34.64250537 33.21366831]
バイアス項なしでの予測結果：
[38.07432601 32.28749487 35.69171582 35.29261206 30.79883648]


バイアス項を導入した場合（最初に動かした結果）と比べると，予測の値が変わっていることが確認できます．
感覚的にはバイアス項を入れた方が良い結果になりそうな気もしますが，**良いか悪いかをパっと見ではなく定量的に評価する必要が本来はあります**．
提出するとスコアが返ってくるので定量的な評価をすることができますが，一日の提出回数が5回に限られているため，一見すると悪そうなものに5回のうちの1回を使ってしまうのはもったいない気もします．
**提出をする前の定量的な評価**についても次回学んでいきましょう．

今日の資料はここで終了です．おつかれさまでした．
上で述べていますが，sklearnには他にも様々なモデル・アルゴリズムが実装されているので，時間があればぜひ色々試してみてください（授業資料は最低限の手引きくらいの気持ちで）．LinearRegressionだけでもあと一つくらいはいじるパラメータがあると思います．
やる気のある方はぜひ金曜日以外も積極的に取り組んでみてください．


## まとめその2
 - 機械学習のアルゴリズムは（基本的には）数値の情報しか使えない．非数値的な情報はどうにかして数的な情報に変換する必要がある．
 - sklearnでは以下の手順で予測モデルを作成，学習しテストデータの予測を行う：(1)インスタンスを作成し，(2)`fit`メソッドでモデルを学習，(3)`predict`メソッドでテストデータの目標値を予測．
 - `np.savetxt`で予測結果をファイルに書き込むことができる．
 - 多くの予測モデルにはユーザが決めるパラメータがあり，それらをハイパーパラメータという．ハイパーパラメータの値によって結果は大きく変わることもあるため，試行錯誤の余地がある．
 - 「文字列の特徴をどうするか」「提出をする前の定量的な評価」については次回を行う ．